# Load Data

In [1]:
import pandas as pd 

# columns correspond to unitigs so we must transpose this table
def load_unitig_data(path) -> pd.DataFrame:
    sr = pd.read_csv(path, sep=' ')
    sr.set_index('pattern_id',inplace=True)
    return sr.T[1:]

azm_sr = load_unitig_data("azm_sr_gwas_filtered_unitigs.Rtab")
cfx_sr = load_unitig_data("cfx_sr_gwas_filtered_unitigs.Rtab")
cip_sr = load_unitig_data("cip_sr_gwas_filtered_unitigs.Rtab")

metadata = pd.read_csv('metadata.csv')
metadata.set_index('Sample_ID',inplace=True)


In [77]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            3536 non-null   float64
 1   Country         3785 non-null   object 
 2   Continent       3785 non-null   object 
 3   Beta.lactamase  1927 non-null   object 
 4   Azithromycin    3480 non-null   object 
 5   Ciprofloxacin   3129 non-null   object 
 6   Ceftriaxone     3436 non-null   object 
 7   Cefixime        3405 non-null   object 
 8   Tetracycline    1472 non-null   object 
 9   Penicillin      1465 non-null   object 
 10  NG_MAST         3779 non-null   object 
 11  Group           3786 non-null   int64  
 12  azm_mic         3478 non-null   float64
 13  cip_mic         3088 non-null   float64
 14  cro_mic         3434 non-null   float64
 15  cfx_mic         3401 non-null   float64
 16  tet_mic         1472 non-null   float64
 17  pen_mic         1465 no

In [2]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


    # Null/NA Cleaning in Labels

1. Remove rows with NaN in the labels we are trying to predict: 'azm_sr','cfx_sr', 'cip_sr'

---

Note to Jacob:
If I remove all rows with nulls, then we drop down to ~1k entries. Not ideal. I will remove only the nulls in the target labels. By doing that I was able to preserve ~2800 entries.

Additionally, I can technically replace the nulls in the feature set with averages whether they are continuous or discrete, but since we are going to be building some kind of predictive model later, it would be bad practice to run column averages in df.fillna() before we split our dataset into training and test sets.

Therefore, I am going to split the dataset into a training and test set first.

What do you think?

-Jacob
We could evaluate how to fill the NaN values based on how the data is skewed for each feature we want to predict. 
if skew > 0 -> fill NaN with mean .. there are more 0s than 1s, mean will reflect the distribution of 0s and 1s
else if skew < 0 -> fill NaN with median .. there are more 1s than 0s, using the median ensures our negatively skewed data wont be affected by an uneven distribution of 0s and 1s when we fill it


In [21]:

def impute_cols_by_skew(df, columns):
    for column_name in columns:
        sr_skew = df[column_name].skew()
        impute_value = None
        if sr_skew > 0:
            impute_value = df[column_name].mean()
        elif sr_skew < 0:
            impute_value = df[column_name].median()
        df[column_name].fillna(impute_value)

#metadata.dropna(axis=0, how='any', inplace=True, subset=['azm_sr', 'cfx_sr', 'cip_sr'])

It looks like they're all positively skewed so we can use mean values, lets try continuous for now and see what happens. It was worth trying this though

In [4]:
metadata.head(5)

,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


# Removing Un-needed Columns
2. Removing 'Year', and unimportant labels 'cro_sr', 'tet_sr', 'pen_sr'

In [5]:
useless_columns = ['Year', 'cro_sr', 'tet_sr', 'pen_sr']

metadata.drop(labels=useless_columns, axis=1, inplace=True)

In [58]:
metadata.columns

Index(['Country', 'Continent', 'Beta.lactamase', 'Azithromycin',
       'Ciprofloxacin', 'Ceftriaxone', 'Cefixime', 'Tetracycline',
       'Penicillin', 'NG_MAST', 'Group', 'azm_mic', 'cip_mic', 'cro_mic',
       'cfx_mic', 'tet_mic', 'pen_mic', 'log2_azm_mic', 'log2_cip_mic',
       'log2_cro_mic', 'log2_cfx_mic', 'log2_tet_mic', 'log2_pen_mic',
       'azm_sr', 'cip_sr', 'cfx_sr'],
      dtype='object')

# Cleaning non-numeric entries in numeric fields to NaN.

3. Turn Non Numeric Entries in Numeric Columns into NaN
4. Cast all numeric rows into float32
---


Notes: Turning them into NaN for now. Will engineer values for all NaNs after train and test splits are made

In [6]:
numeric_columns = [
'Azithromycin',
'Ciprofloxacin',
'Ceftriaxone',
'Cefixime',
'Tetracycline',
'Penicillin',
'NG_MAST',
'Group',
'azm_mic',
'cip_mic',
'cro_mic',
'cfx_mic',
'tet_mic',
'pen_mic',
'log2_azm_mic',
'log2_cip_mic',
'log2_cro_mic',
'log2_cfx_mic',
'log2_tet_mic',
'log2_pen_mic',
'azm_sr',
'cip_sr',
'cfx_sr',
]

for column in numeric_columns:
    metadata[column] = pd.to_numeric(metadata[column], errors='coerce', downcast="float") #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

In [7]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         3785 non-null   object 
 1   Continent       3785 non-null   object 
 2   Beta.lactamase  1927 non-null   object 
 3   Azithromycin    3373 non-null   float32
 4   Ciprofloxacin   3069 non-null   float32
 5   Ceftriaxone     3432 non-null   float32
 6   Cefixime        3366 non-null   float32
 7   Tetracycline    1470 non-null   float32
 8   Penicillin      1461 non-null   float32
 9   NG_MAST         3238 non-null   float32
 10  Group           3786 non-null   float32
 11  azm_mic         3478 non-null   float32
 12  cip_mic         3088 non-null   float32
 13  cro_mic         3434 non-null   float32
 14  cfx_mic         3401 non-null   float32
 15  tet_mic         1472 non-null   float32
 16  pen_mic         1465 non-null   float32
 17  log2_azm_mic    3478 no

In [8]:
metadata.head(10)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,UK,Europe,NaN,NaN,NaN,0.016,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549290,UK,Europe,NaN,NaN,NaN,0.004,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549291,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549287,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549288,UK,Europe,NaN,NaN,NaN,0.008,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549299,UK,Europe,NaN,NaN,NaN,0.012,NaN,NaN,NaN,NaN,...,NaN,9.0,NaN,-6.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549292,UK,Europe,NaN,NaN,NaN,0.023,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-5.442222,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549298,UK,Europe,NaN,0.5,NaN,0.094,NaN,NaN,NaN,NaN,...,NaN,-1.0,NaN,-3.411196,NaN,NaN,NaN,0.0,NaN,NaN
ERR1549296,UK,Europe,NaN,0.5,NaN,0.094,NaN,NaN,NaN,NaN,...,NaN,-1.0,NaN,-3.411196,NaN,NaN,NaN,0.0,NaN,NaN


# One Hot Encode Categorical Columns

5. Turn Categorical Location entries into numerical representation https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html
6. Handle 'Beta.lactamase' special case, as it is discrete with many NaNs. Cannot be engineered in the same way as continuous features. One hot encode like the other discrete features but set to all 0 if NaN

In [86]:
geographic_columns=['Country', 'Continent']
metadata = pd.get_dummies(data=metadata, prefix="Encoded", columns=geographic_columns, dtype=float)
metadata = pd.get_dummies(data=metadata, prefix="Encoded_Beta.lactamase", columns=['Beta.lactamase'], dtype=float)


In [9]:
metadata.head(5)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,UK,Europe,NaN,NaN,NaN,0.016,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549290,UK,Europe,NaN,NaN,NaN,0.004,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549291,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549287,UK,Europe,NaN,NaN,NaN,0.006,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,NaN
ERR1549288,UK,Europe,NaN,NaN,NaN,0.008,NaN,NaN,NaN,9768.0,...,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,NaN


In [10]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3786 entries, ERR1549286 to ERR2172354
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         3785 non-null   object 
 1   Continent       3785 non-null   object 
 2   Beta.lactamase  1927 non-null   object 
 3   Azithromycin    3373 non-null   float32
 4   Ciprofloxacin   3069 non-null   float32
 5   Ceftriaxone     3432 non-null   float32
 6   Cefixime        3366 non-null   float32
 7   Tetracycline    1470 non-null   float32
 8   Penicillin      1461 non-null   float32
 9   NG_MAST         3238 non-null   float32
 10  Group           3786 non-null   float32
 11  azm_mic         3478 non-null   float32
 12  cip_mic         3088 non-null   float32
 13  cro_mic         3434 non-null   float32
 14  cfx_mic         3401 non-null   float32
 15  tet_mic         1472 non-null   float32
 16  pen_mic         1465 non-null   float32
 17  log2_azm_mic    3478 no

# Split Dataframe into Train and Test

In [13]:
from sklearn.model_selection import train_test_split
train_inputs,test_inputs = train_test_split(metadata, test_size=0.4, random_state=42)   # partition and mix entries
print(train_inputs.size,":", test_inputs.size)

59046 : 39390


In [23]:
# apply skew based imputation

targets = ['azm_sr','cfx_sr','cip_sr']
impute_cols_by_skew(train_inputs, targets)
impute_cols_by_skew(test_inputs, targets)

train_inputs.head(5)


,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
15335_5#74,USA,America,S,1.000,8.000,0.015,NaN,NaN,NaN,1407.0,...,NaN,0.000000,3.000000,-6.058894,NaN,NaN,NaN,0.0,1.000000,0.001471
16853_3#19,Italy,Europe,NaN,0.094,0.004,0.002,0.016,NaN,NaN,10676.0,...,NaN,-3.411196,-7.965784,-8.965784,-5.965784,NaN,NaN,0.0,0.000000,0.000000
10625_6#18,Japan,Asia,S,0.750,32.000,0.094,0.190,2.0,2.0,7328.0,...,2.0,-0.415038,5.000000,-3.411196,-2.395929,1.0,1.0,0.0,1.000000,0.000000
10_062,Greece,Europe,NaN,16.000,NaN,0.047,0.125,NaN,NaN,3806.0,...,NaN,4.000000,NaN,-4.411195,-3.000000,NaN,NaN,1.0,0.457735,0.000000
8727_5#86,USA,America,S,0.500,32.000,0.030,0.060,2.0,2.0,1978.0,...,2.0,-1.000000,5.000000,-5.058894,-4.058894,1.0,1.0,0.0,1.000000,0.000000


# Normalize Numerical Features 

In [25]:
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

train_inputs[numeric_columns] = normalizer.fit_transform(train_inputs[numeric_columns])
test_inputs[numeric_columns] = normalizer.fit_transform(test_inputs[numeric_columns])

train_inputs.head(5)

,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,...,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cfx_sr
Sample_ID,,,,,,,,,,,,,,,,,,,,,
SRR3360667,UK,Europe,NaN,0.014640,0.499984,NaN,NaN,0.031220,0.062471,0.009818,...,0.062471,0.248393,0.937366,NaN,NaN,0.665905,0.732724,0.0,1.00000,0.001468
8727_8#71,USA,America,S,0.030280,0.000203,0.014754,0.062266,0.003875,0.007781,NaN,...,0.007781,0.311027,0.244704,0.493648,0.665714,0.465447,0.532266,0.0,0.00000,0.000000
SRR2736190,Canada,America,NaN,0.499499,NaN,0.001750,0.000750,NaN,NaN,0.202451,...,NaN,0.561562,NaN,0.250715,0.167143,NaN,NaN,1.0,0.46976,0.000000
17176_1#77,USA,America,S,0.014640,0.000094,0.014754,0.031008,NaN,NaN,0.247275,...,NaN,0.248393,0.187902,0.493648,0.582142,NaN,NaN,0.0,0.00000,0.000000
10356_1#2,Caribbean,America,R,0.000438,0.000000,0.000750,0.003751,0.001453,0.093722,NaN,...,0.093722,0.032793,0.062634,0.167143,0.334287,0.372173,0.771810,0.0,0.00000,0.000000


# Re-engineer Numerical NaN features if possible

### Data Stats

In [88]:
j=[0,0,0]
samples = metadata.index
for sample in samples:
    if metadata['azm_sr'][sample]:
        j[0] +=1
    if metadata['cfx_sr'][sample]:
        j[1] +=1
    if metadata['cip_sr'][sample]:
        j[2] +=1

print(j[0]/len(samples), "% of samples have resistance to azm")
print(j[1]/len(samples), "% of samples have resistance to cfx")
print(j[2]/len(samples), "% of samples have resistance to cip")


0.05650319829424307 % of samples have resistance to azm
0.0017768301350390902 % of samples have resistance to cfx
0.4541577825159915 % of samples have resistance to cip


In [89]:
samples = azm_sr.index

# some random unitig from azm_sr

from random import randint
randomUnitig = azm_sr.columns[randint(0,azm_sr.shape[1])]
j = 0
print(azm_sr.shape)
for i in range(azm_sr.shape[0]):
    if azm_sr[randomUnitig][samples[i]]:
        j += 1

print(randomUnitig,"\npresent in", j/azm_sr.shape[0],"% of azm_sr samples (",j,"/",azm_sr.shape[0],')' )

(3970, 515)
ACCCTGGACGCCGGCTACCGCTACCACAACT 
present in 0.9954659949622167 % of azm_sr samples ( 3952 / 3970 )
